In [1]:
import redis
import json

# Define connection variables
host = 'localhost'
port =  6379
password = None # 'password'

# Connect to Redis
r = redis.Redis(host=host, port=port, password=password, decode_responses=True)
print('Connected to Redis')

r.flushdb()

Connected to Redis


True

# Productivity Tracking

## Description of the use case
This use case involves tracking employee activities and productivity using Redis Bitmaps. By recording activity data, businesses can monitor productivity levels and identify areas for improvement.

## Redis Data Structures Used
- **Bitmaps**: To track employee activities and productivity.
- **Hashes**: To store additional information about employees.


In [2]:
# Mock data for the use case
employee_data = {
    'employee_1001': {'name': 'Alice', 'position': 'Developer'},
    'employee_1002': {'name': 'Bob', 'position': 'Designer'},
    'employee_1003': {'name': 'Charlie', 'position': 'Manager'},
    'employee_1004': {'name': 'David', 'position': 'Developer'}
}

# Use Hashes to store employee information
for employee_id, data in employee_data.items():
    r.hset(employee_id, mapping=data)

# Use Bitmaps to track employee activity (e.g., working hours in a week)
# Assume 1 bit per hour, 8 hours per day, 5 days a week = 40 bits per week
# 1 indicates the employee was active during that hour, 0 indicates inactivity
activity_data = {
    'employee_1001': '11111111' * 5,  # Active for all 40 hours
    'employee_1002': '11110000' * 5,  # Active for 4 hours each day
    'employee_1003': '11000000' * 5,  # Active for 2 hours each day
    'employee_1004': '11111110' * 5   # Active for 7 hours each day
}

for employee_id, activity in activity_data.items():
    for i, bit in enumerate(activity):
        r.setbit(f'activity:{employee_id}', i, int(bit))

# Function to calculate productivity
def calculate_productivity(employee_id):
    return r.bitcount(f'activity:{employee_id}')

# Calculate and print productivity for each employee
for employee_id in employee_data.keys():
    productivity = calculate_productivity(employee_id)
    print(f'Productivity for {employee_id}: {productivity} hours')

# Print the employee information
for employee_id in employee_data.keys():
    print(f"{employee_id}: {r.hgetall(employee_id)}")


Productivity for employee_1001: 40 hours
Productivity for employee_1002: 20 hours
Productivity for employee_1003: 10 hours
Productivity for employee_1004: 35 hours
employee_1001: {'name': 'Alice', 'position': 'Developer'}
employee_1002: {'name': 'Bob', 'position': 'Designer'}
employee_1003: {'name': 'Charlie', 'position': 'Manager'}
employee_1004: {'name': 'David', 'position': 'Developer'}
